In [74]:
import pandas as pd

# Cargar los archivos CSV
biogrid_data = pd.read_csv('Biogrid_data_bacillus_subtilis.csv')
intact_data = pd.read_csv('Intact_data_bacillus_subtilis.csv')
string_data = pd.read_csv('String_bacillus_subtilis_CW14.csv')
deseq_data = pd.read_csv('deSeqRes_combined_with_GO_Biological_Process_terms_sorted.csv')

# Renombrar las columnas del archivo de Biogrid
biogrid_data = biogrid_data.rename(columns={
    "Official symbol for Interactor A": "Interactor A Genes",
    "Official symbol for Interactor B": "Interactor B Genes"
})

# Renombrar las columnas del archivo de String
biogrid_data = string_data.rename(columns={
    "Organism ID for Interactor A": "Taxid interactor A",
    "Organism ID for Interactor B": "Taxid interactor B"
})

# Seleccionar solo las columnas de interés y eliminar las columnas especificadas
biogrid_data = biogrid_data[["Interactor A Genes", "Interactor B Genes", "Taxid interactor A", "Taxid interactor B", "Genes"]]
intact_data = intact_data[["Interactor A Genes", "Interactor B Genes", "Taxid interactor A", "Taxid interactor B", "Genes"]]
string_data = string_data[["Interactor A Genes", "Interactor B Genes", "Taxid interactor A", "Taxid interactor B", "Genes"]]

# Realizar el merge de los archivos en un solo DataFrame
merged_data = pd.merge(biogrid_data, intact_data, on=['Genes', 'Interactor A Genes', 'Interactor B Genes', 'Taxid interactor A', 'Taxid interactor B'], how='outer')
merged_data = pd.merge(merged_data, string_data, on=['Genes', 'Interactor A Genes', 'Interactor B Genes', 'Taxid interactor A', 'Taxid interactor B'], how='outer')

# Eliminar duplicados
merged_data = merged_data.drop_duplicates()

# Lista de genes para filtrar
genes_to_filter = [
    "CCL4", "CSF2", "CSF3", "CXCL8", "CXCL10", "CCL5", "TNF", "PLAU", "LTB", 
    "CX3CL1", "CXCL2", "TNFSF14", "CXCL5", "CSF1", "CXCL11", "CXCL1", "CYP1B1", 
    "CCL2", "BIRC3", "CCL22", "TNFAIP3", "NFKBIZ", "NANOS3", "RCAN1", "RGS2"
]

# Filtrar el DataFrame por los genes especificados
filtered_data = merged_data[merged_data['Genes'].isin(genes_to_filter)]

# Mostrar las primeras filas del DataFrame filtrado para verificar
print(filtered_data.head())

# Manejar la información de deseq_data por separado
deseq_data_filtered = deseq_data[deseq_data['Genes'].isin(genes_to_filter)]

# Mostrar las primeras filas del DataFrame de deseq_data filtrado para verificar
print(deseq_data_filtered.head())

  Interactor A Genes Interactor B Genes Taxid interactor A Taxid interactor B  \
0              CCL20               CCL4       Homo sapiens       Homo sapiens   
1               CCL4             CXCL10       Homo sapiens       Homo sapiens   
2               CSF2               CCL4       Homo sapiens       Homo sapiens   
3               CSF3               CCL4       Homo sapiens       Homo sapiens   
4               CCL4             CX3CL1       Homo sapiens       Homo sapiens   

  Genes  
0  CCL4  
1  CCL4  
2  CCL4  
3  CCL4  
4  CCL4  
    Genes                                               Term  \
0    CCL4  Inflammatory Response (GO:0006954), Cytokine-M...   
1    CSF2  Cytokine-Mediated Signaling Pathway (GO:001922...   
2    CSF3  Cytokine-Mediated Signaling Pathway (GO:001922...   
3   CXCL8  Inflammatory Response (GO:0006954), Cytokine-M...   
4  CXCL10  Inflammatory Response (GO:0006954), Cytokine-M...   

                                    Adjusted P-value  Log2FoldChange

In [75]:
import pandas as pd

# Cargar los archivos CSV
#filtered_data = pd.read_csv('filtered_data.csv')  # Asegúrate de tener este archivo generado previamente
data_with_fc = pd.read_csv('deSeqRes_filtrado_bacillus_subtilis_CW14_with_FC.csv')
data_with_terms = pd.read_csv('deSeqRes_combined_with_GO_Biological_Process_terms_sorted.csv')

# Renombrar las columnas del archivo de Fold Change
data_with_fc = data_with_fc.rename(columns={
    'GeneSymbol': 'Genes',
    'DirectionalFoldChange': 'FoldChange'
})

# Crear una nueva columna 'Regulation' basada en el signo de 'FoldChange'
data_with_fc['Regulation'] = data_with_fc['FoldChange'].apply(lambda x: 'UpRegulated' if x > 0 else 'DownRegulated')

# Tomar el valor absoluto de 'FoldChange'
data_with_fc['FoldChange'] = data_with_fc['FoldChange'].abs()

# Lista de genes para filtrar
genes_to_filter = [
    "CCL4", "CSF2", "CSF3", "CXCL8", "CXCL10", "CCL5", "TNF", "PLAU", "LTB", 
    "CX3CL1", "CXCL2", "TNFSF14", "CXCL5", "CSF1", "CXCL11", "CXCL1", "CYP1B1", 
    "CCL2", "BIRC3", "CCL22", "TNFAIP3", "NFKBIZ", "NANOS3", "RCAN1", "RGS2"
]

# Filtrar data_with_fc por los genes especificados
data_with_fc_filtered = data_with_fc[data_with_fc['Genes'].isin(genes_to_filter)]

# Filtrar data_with_terms por los genes especificados
data_with_terms_filtered = data_with_terms[data_with_terms['Genes'].isin(genes_to_filter)]

# Crear listas para nodos y aristas
nodos = set()
aristas = []

# Iterar sobre cada fila del DataFrame filtered_data para crear nodos y aristas
for index, row in filtered_data.iterrows():
    gene = row['Genes']
    if gene in genes_to_filter:
        nodos.add((gene, 'Gene'))  # Agregar el gen como nodo

        # Agregar interacciones como aristas
        interactor_a = row['Interactor A Genes']
        interactor_b = row['Interactor B Genes']
        taxid_a = row['Taxid interactor A']
        taxid_b = row['Taxid interactor B']
        if pd.notna(interactor_a):
            aristas.append((gene, interactor_a, 'pp', taxid_a, taxid_b))
            nodos.add((interactor_a, 'Gene'))  # Agregar interactor A como nodo
        if pd.notna(interactor_b):
            aristas.append((gene, interactor_b, 'pp', taxid_a, taxid_b))
            nodos.add((interactor_b, 'Gene'))  # Agregar interactor B como nodo

# Asegurarse de que todos los genes en genes_to_filter estén en nodos
for gene in genes_to_filter:
    nodos.add((gene, 'Gene'))

# Crear DataFrame de nodos
nodos_df = pd.DataFrame(list(nodos), columns=['Id', 'Type'])

# Realizar el merge para completar la información faltante con data_with_fc_filtered
nodos_df = pd.merge(nodos_df, data_with_fc_filtered[['Genes', 'FoldChange', 'Regulation']], left_on='Id', right_on='Genes', how='left')

# Realizar el merge para agregar la información de términos con data_with_terms_filtered
nodos_df = pd.merge(nodos_df, data_with_terms_filtered[['Genes', 'Term']], left_on='Id', right_on='Genes', how='left')

# Eliminar la columna 'Genes' después del merge, si existe
if 'Genes' in nodos_df.columns:
    nodos_df = nodos_df.drop(columns=['Genes'])

# Eliminar duplicados en nodos_df
nodos_df = nodos_df.drop_duplicates()

# Filtrar el DataFrame de nodos por los genes especificados
nodos_df = nodos_df[nodos_df['Id'].isin(genes_to_filter)]

# Eliminar las columnas Genes_x y Genes_y si existen
columns_to_drop = ['Genes_x', 'Genes_y']
nodos_df = nodos_df.drop(columns=[col for col in columns_to_drop if col in nodos_df.columns])

# Guardar el archivo de nodos actualizado
nodos_df.to_csv('nodos.csv', index=False)

# Crear DataFrame de aristas con información adicional
aristas_df = pd.DataFrame(aristas, columns=['Source', 'Target', 'Interaction', 'Taxid interactor A', 'Taxid interactor B'])

# Eliminar duplicados en aristas
aristas_df = aristas_df.drop_duplicates()

# Guardar el archivo de aristas
aristas_df.to_csv('aristas.csv', index=False)

# Mostrar las primeras filas del DataFrame de nodos actualizado para verificar
print(nodos_df.head())

# Mostrar las primeras filas del DataFrame de aristas actualizado para verificar
print(aristas_df.head())

        Id  Type  FoldChange   Regulation  \
0     CSF2  Gene   32.563351  UpRegulated   
1    CXCL8  Gene   25.041102  UpRegulated   
2   CXCL10  Gene   20.255812  UpRegulated   
3  TNFSF14  Gene    7.732854  UpRegulated   
5   NFKBIZ  Gene    4.869511  UpRegulated   

                                                Term  
0  Cytokine-Mediated Signaling Pathway (GO:001922...  
1  Inflammatory Response (GO:0006954), Cytokine-M...  
2  Inflammatory Response (GO:0006954), Cytokine-M...  
3  Positive Regulation Of Intracellular Signal Tr...  
5  Positive Regulation Of Response To External St...  
  Source  Target Interaction Taxid interactor A Taxid interactor B
0   CCL4   CCL20          pp       Homo sapiens       Homo sapiens
1   CCL4    CCL4          pp       Homo sapiens       Homo sapiens
3   CCL4  CXCL10          pp       Homo sapiens       Homo sapiens
4   CCL4    CSF2          pp       Homo sapiens       Homo sapiens
6   CCL4    CSF3          pp       Homo sapiens       Homo sapie

In [2]:
import pandas as pd

# Cargar los archivos CSV
data_with_fc = pd.read_csv('deSeqRes_filtrado_bacillus_subtilis_CW14_with_FC.csv')
data_with_terms = pd.read_csv('deSeqRes_combined_with_GO_Biological_Process_terms_sorted.csv')

# Renombrar las columnas del archivo de Fold Change
data_with_fc = data_with_fc.rename(columns={
    'GeneSymbol': 'Genes',
    'DirectionalFoldChange': 'FoldChange'
})

# Crear una nueva columna 'Regulation' basada en el signo de 'FoldChange'
data_with_fc['Regulation'] = data_with_fc['FoldChange'].apply(lambda x: 'UpRegulated' if x > 0 else 'DownRegulated')

# Tomar el valor absoluto de 'FoldChange'
data_with_fc['FoldChange'] = data_with_fc['FoldChange'].abs()

# Lista de genes para filtrar
genes_to_filter = [
    "CCL4", "CSF2", "CSF3", "CXCL8", "CXCL10", "CCL5", "TNF", "PLAU", "LTB", 
    "CX3CL1", "CXCL2", "TNFSF14", "CXCL5", "CSF1", "CXCL11", "CXCL1", "CYP1B1", 
    "CCL2", "BIRC3", "CCL22", "TNFAIP3", "NFKBIZ", "NANOS3", "RCAN1", "RGS2"
]

# Filtrar data_with_terms por los genes especificados
data_with_terms_filtered = data_with_terms[data_with_terms['Genes'].isin(genes_to_filter)]

# Dividir los valores de "Term" por comas y expandir los términos para que cada término tenga su propia fila
data_with_terms_expanded = data_with_terms_filtered.assign(Term=data_with_terms_filtered['Term'].str.split(',')).explode('Term')

# Eliminar espacios en blanco alrededor de los términos
data_with_terms_expanded['Term'] = data_with_terms_expanded['Term'].str.strip()

# Seleccionar solo las columnas 'Genes' y 'Term'
data_with_terms_expanded = data_with_terms_expanded[['Genes', 'Term']]

# Guardar el resultado en un nuevo archivo CSV
data_with_terms_expanded.to_csv('genes_with_terms_expanded.csv', index=False)

# Mostrar las primeras filas del DataFrame expandido para verificar
print(data_with_terms_expanded.head())

  Genes                                               Term
0  CCL4                 Inflammatory Response (GO:0006954)
0  CCL4   Cytokine-Mediated Signaling Pathway (GO:0019221)
0  CCL4  Chemokine-Mediated Signaling Pathway (GO:0070098)
0  CCL4        Cellular Response To Chemokine (GO:1990869)
0  CCL4                 Neutrophil Chemotaxis (GO:0030593)


In [4]:
import pandas as pd

# Cargar los archivos existentes
nodes_df = pd.read_csv('nodos.csv')
edges_df = pd.read_csv('aristas.csv')
genes_with_terms_expanded = pd.read_csv('genes_with_terms_expanded.csv')

# Eliminar la columna 'Term' de nodes_df ya que no es necesaria
if 'Term' in nodes_df.columns:
    nodes_df = nodes_df.drop(columns=['Term'])

# Asegurarse de que cada término en genes_with_terms_expanded esté presente como un nodo en nodes.csv
terms = genes_with_terms_expanded['Term'].unique()
new_nodes = pd.DataFrame(terms, columns=['Id'])
new_nodes['Type'] = 'Term'

# Concatenar los nuevos nodos con los existentes y eliminar duplicados
nodes_df = pd.concat([nodes_df, new_nodes]).drop_duplicates(subset=['Id'])

# Crear nuevas aristas que representen las relaciones entre los genes y los términos
new_edges = genes_with_terms_expanded.rename(columns={'Genes': 'Source', 'Term': 'Target'})
new_edges['Interaction'] = 'association'
new_edges['Taxid interactor A'] = ''
new_edges['Taxid interactor B'] = ''

# Concatenar las nuevas aristas con las existentes y eliminar duplicados
edges_df = pd.concat([edges_df, new_edges]).drop_duplicates()

# Guardar los archivos actualizados
nodes_df.to_csv('nodes.csv', index=False)
edges_df.to_csv('edges.csv', index=False)

# Mostrar las primeras filas de los DataFrames actualizados para verificar
print(nodes_df.head())
print(edges_df.head())

        Id  Type  FoldChange   Regulation
0     CSF2  Gene   32.563351  UpRegulated
1    CXCL8  Gene   25.041102  UpRegulated
2   CXCL10  Gene   20.255812  UpRegulated
3  TNFSF14  Gene    7.732854  UpRegulated
4   NFKBIZ  Gene    4.869511  UpRegulated
  Source  Target Interaction Taxid interactor A Taxid interactor B
0   CCL4   CCL20          pp       Homo sapiens       Homo sapiens
1   CCL4    CCL4          pp       Homo sapiens       Homo sapiens
2   CCL4  CXCL10          pp       Homo sapiens       Homo sapiens
3   CCL4    CSF2          pp       Homo sapiens       Homo sapiens
4   CCL4    CSF3          pp       Homo sapiens       Homo sapiens
